## Quantidade de SKU por loja

In [ ]:
import pandas as pd
import numpy as np
import os
import sys
import datetime
import json


project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Froms
from src.gcpUtils.auth import getCredentials
from src.gcpUtils.bigQuery import pandasToBq, tableToPandas
from src.gcpUtils.google_storage_manager import *

cred = getCredentials("../bd/planejamento-animale-292719-296d49ccdea6.json")

In [ ]:


# --- 1. Configurações Iniciais ---

ARQUIVO_JSON_FILIAIS = '../dados/filiais_inferior_30.json'
DATA_ANALISE = '2025-01-15'  # Formato AAAA-MM-DD
TABELA_BIGQUERY = 'planejamento-animale-292719.checklists_rollout.ANIMALE_checklist' # IMPORTANTE: Ajuste isso

print("pasta atual:", os.getcwd())
# Caminho para o arquivo de credenciais
# (Ajustei o caminho baseado na sua imagem: subindo de 'Notebooks' para 'bd')
PATH_CREDENCAIS = '../bd/planejamento-animale-292719-296d49ccdea6.json'

# --- 2. Carregar Filiais do JSON ---

try:
    print(f"Carregando filiais do arquivo: {ARQUIVO_JSON_FILIAIS}...")
    
    with open(ARQUIVO_JSON_FILIAIS, 'r', encoding='utf-8') as f:
        dados_filiais = json.load(f)
        
    df_filiais = pd.DataFrame(dados_filiais)
    
    # Pega a lista de nomes únicos de filiais
    lista_nomes_filiais = df_filiais['FILIAL'].unique().tolist()
    
    if not lista_nomes_filiais:
        print("Atenção: Nenhuma filial foi encontrada no arquivo JSON.")
    else:
        print(f"Sucesso. {len(lista_nomes_filiais)} filiais únicas encontradas.")

except FileNotFoundError:
    print(f"Erro: Arquivo JSON não encontrado em '{ARQUIVO_JSON_FILIAIS}'")
    lista_nomes_filiais = []
except Exception as e:
    print(f"Erro ao ler o arquivo JSON: {e}")
    lista_nomes_filiais = []


# --- 3. Montar e Executar a Query ---

if lista_nomes_filiais:
    
    # Ex: Transforma ['Filial A', 'Filial B'] em "('Filial A', 'Filial B')"
    filiais_para_query = "','".join(lista_nomes_filiais)
    filiais_para_query = f"('{filiais_para_query}')"
    
    query = f"""
    SELECT 
        FILIAL, 
        COUNT(DISTINCT SKU) as QTD_SKU
    FROM 
        `{TABELA_BIGQUERY}`
    WHERE 
        DATE(DATA) = '{DATA_ANALISE}' 
    
        AND FILIAL IN {filiais_para_query}
    GROUP BY 
        FILIAL
    ORDER BY
        QTD_SKU DESC
    """
    
    try:
        # --- 4. Buscar os dados ---
        
        print("Executando consulta no BigQuery...")
        df_resultado = tableToPandas(query, 'planejamento-animale-292719', cred)
        
        print("\n--- Resultado da Consulta ---")
        if df_resultado.empty:
            print("A consulta não retornou dados.")
        else:
            print(df_resultado)

    except Exception as e:
        print(f"\nErro ao executar a consulta no BigQuery: {e}")

else:
    print("\nAnálise não executada pois nenhuma filial foi carregada.")